In [99]:
import numpy as np

def generate_training_data(num_samples=105, c=4):
    X_train = []
    y_train = []

    # Define probabilities for winding numbers
    probabilities = [0.13, 0.17, 0.20, 0.32, 0.18]
    for _ in range(num_samples):
        # Generate random coefficients for Fourier series
        a_x = np.random.uniform(-1, 1, size=c+1)
        b_x = np.random.uniform(-1, 1, size=c+1)
        a_y = np.random.uniform(-1, 1, size=c+1)
        b_y = np.random.uniform(-1, 1, size=c+1)

        # Generate winding number
        w = np.random.choice([-2, -1, 0, 1, 2], p=probabilities)

        # Append Hamiltonian and winding number to training data
        X_train.append((a_x, b_x, a_y, b_y))
        y_train.append(w)

    return np.array(X_train), np.array(y_train)

# Example usage
X_train, y_train = generate_training_data(num_samples=1000)

In [100]:
X_train.shape, y_train.shape

((1000, 4, 5), (1000,))

In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Define the fully connected network architecture
model = Sequential([
    Dense(40, activation='relu', input_shape=(20,)),  # First hidden layer with 40 neurons
    Dense(32, activation='relu'),  # Second hidden layer with 32 neurons
    Dense(2, activation='linear')  # Output layer with 2 neurons
])

# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 40)                840       
                                                                 
 dense_4 (Dense)             (None, 32)                1312      
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 2218 (8.66 KB)
Trainable params: 2218 (8.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [103]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split


# Clear the previous TensorFlow session
tf.keras.backend.clear_session()

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train_flat = np.array([sample.flatten().reshape(-1) for sample in X_train])

X_test_flat = np.array([sample.flatten().reshape(-1) for sample in X_test])

# Compile the model
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_flat, y_train, epochs=50, batch_size=32, validation_data=(X_test_flat, y_test))

Epoch 1/50
20/20 [==============================] - 1s 15ms/step - loss: 1.8813 - accuracy: 0.2484 - val_loss: 1.4786 - val_accuracy: 0.3125
Epoch 2/50
20/20 [==============================] - 0s 4ms/step - loss: 1.7870 - accuracy: 0.2750 - val_loss: 1.4555 - val_accuracy: 0.3187
Epoch 3/50
20/20 [==============================] - 0s 4ms/step - loss: 1.7480 - accuracy: 0.2734 - val_loss: 1.4499 - val_accuracy: 0.3000
Epoch 4/50
20/20 [==============================] - 0s 6ms/step - loss: 1.7233 - accuracy: 0.2812 - val_loss: 1.4518 - val_accuracy: 0.2875
Epoch 5/50
20/20 [==============================] - 0s 6ms/step - loss: 1.7042 - accuracy: 0.2719 - val_loss: 1.4572 - val_accuracy: 0.3063
Epoch 6/50
20/20 [==============================] - 0s 5ms/step - loss: 1.6862 - accuracy: 0.2625 - val_loss: 1.4626 - val_accuracy: 0.3125
Epoch 7/50
20/20 [==============================] - 0s 4ms/step - loss: 1.6675 - accuracy: 0.2562 - val_loss: 1.4643 - val_accuracy: 0.3125
Epoch 8/50
20/20 [=

In [104]:
# Evaluate the model on the training data
loss, accuracy = model.evaluate(X_train_flat, y_train)
print("Training Loss:", loss)
print("Training Accuracy:", accuracy)

# Evaluate the model on the test data (assuming you have X_test_flat and y_test)
loss, accuracy = model.evaluate(X_test_flat, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


20/20 [==============================] - 0s 8ms/step - loss: 0.7092 - accuracy: 0.2328
Training Loss: 0.7092471718788147
Training Accuracy: 0.23281249403953552
5/5 [==============================] - 0s 5ms/step - loss: 2.1938 - accuracy: 0.3625
Test Loss: 2.1938490867614746
Test Accuracy: 0.36250001192092896
